In [10]:
import torch
import os
from PIL import Image
import clip
import os.path as osp
import os, sys
import torchvision.utils as vutils
sys.path.insert(0, '../')

from lib.utils import load_model_weights,mkdir_p
from models.GALIP import NetG, CLIP_TXT_ENCODER

In [11]:
import torch

# Print PyTorch version
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("Is CUDA available:", cuda_available)

# Print CUDA version if available
if cuda_available:
    print("CUDA version:", torch.version.cuda)


PyTorch version: 1.9.1+cu111
Is CUDA available: True
CUDA version: 11.1


In [12]:
device = 'cuda:0' # 'cpu' # 'cuda:0'
CLIP_text = "ViT-B/32"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model = clip_model.eval()

In [13]:
text_encoder = CLIP_TXT_ENCODER(clip_model).to(device)
netG = NetG(64, 100, 512, 256, 3, False, clip_model).to(device)


In [14]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is CCF8-FC69

 Directory of c:\Users\haitham.alhaji\Desktop\GANCLIP\code\src

2024-07-23  12:25 AM    <DIR>          .
2024-07-22  09:25 PM    <DIR>          ..
2024-07-22  08:24 PM                 0 readme.txt
2024-07-22  11:51 PM             4,971 sample.ipynb
2024-07-23  12:25 AM    <DIR>          samples
2024-07-22  09:05 PM     1,157,700,139 state_epoch_020.pth
2024-07-22  09:05 PM     1,157,700,139 state_epoch_180_birds.pth
2024-07-22  08:24 PM             5,162 test.py
2024-07-22  08:24 PM             8,727 train.py
               6 File(s)  2,315,419,138 bytes
               3 Dir(s)  37,040,152,576 bytes free


In [15]:
path = 'state_epoch_020.pth'
checkpoint = torch.load(path, map_location=torch.device('cuda:0'))
netG = load_model_weights(netG, checkpoint['model']['netG'], multi_gpus=False)

In [16]:
batch_size = 8
noise = torch.randn((batch_size, 100)).to(device)

In [17]:
mkdir_p('./samples')

In [18]:
captions = ['bird swimming']
# generate from text
with torch.no_grad():
    for i in range(len(captions)):
        caption = captions[i]
        tokenized_text = clip.tokenize([caption]).to(device)
        sent_emb, word_emb = text_encoder(tokenized_text)
        sent_emb = sent_emb.repeat(batch_size,1)
        fake_imgs = netG(noise,sent_emb,eval=True).float()
        name = f'{captions[i].replace(" ", "-")}'
        vutils.save_image(fake_imgs.data, './samples/%s.png'%(name), nrow=8, value_range=(-1, 1), normalize=True)